# LlamaIndex와 AgentCore Memory - Medical Knowledge Assistant (장기 메모리)

## 소개

이 노트북은 Amazon Bedrock AgentCore Memory 기능을 LlamaIndex와 통합하여 여러 환자 상담 및 의료 사례에 걸쳐 **장기 메모리** 지속성을 갖춘 Medical Knowledge Assistant를 생성하는 방법을 보여줍니다. 이를 통해 어시스턴트가 누적 의료 지식을 구축하고 수개월 및 수년에 걸쳐 환자 치료를 추적할 수 있습니다.

## 아키텍처 개요

![LlamaIndex AgentCore Long-Term Memory Architecture](LlamaIndex-AgentCore-LTM-Arch.png)

## 튜토리얼 세부 정보

**튜토리얼 세부 정보:**
- **튜토리얼 유형**: 장기 크로스 세션 메모리
- **Agent 사용 사례**: Medical Knowledge Assistant
- **Agentic Framework**: LlamaIndex
- **LLM model**: Anthropic Claude 3.7 Sonnet
- **튜토리얼 구성 요소**: AgentCore 장기 메모리, LlamaIndex Agent, Medical Tools
- **예제 복잡도**: 고급

## 비즈니스 가치

**엔터프라이즈 의료 인텔리전스**: 환자 지식을 축적하고, 치료 진화를 추적하며, 사례 및 기간에 걸쳐 포괄적인 의료 기록을 유지하는 지속적인 AI 메모리로 의료 실무를 혁신하세요.

**주요 전문적 이점:**
- **환자 연속성**: 의료 진료 및 치료 팀 간 원활한 지식 전달
- **임상 메모리**: 중요한 환자 기록, 치료 및 결과를 영구적으로 보존
- **환자 간 인텔리전스**: 여러 환자 사례에 걸쳐 패턴 및 연결 식별
- **치료 우수성**: 우수한 치료 결정을 위해 과거 환자 데이터 활용
- **인구 건강**: 종단적 환자 치료를 위한 상세한 컨텍스트 유지
- **품질 개선**: 시간 경과에 따른 치료 프로토콜 및 효과 추적

## 장기 메모리 구성

**기술 설정**: 이 튜토리얼은 12개월 보존을 위한 Semantic Strategy와 함께 AgentCore Memory를 사용합니다:
- **메모리 유형**: 자동 인사이트 추출을 사용하는 Semantic strategy
- **보존**: 환자 치료 연속성을 위한 365일 이벤트 만료
- **크로스 세션**: 동일한 actor_id + memory_id, 의료 기간별로 다른 session_id
- **검색 기능**: 환자 기록 전체에 대한 시맨틱 검색을 위한 내장 메모리 검색 도구

## 기술 개요

**주요 장기 메모리 구성 요소:**
1. **Semantic Strategy 구성**: 365일 보존과 함께 자동 인사이트 추출을 위해 SemanticStrategy 사용
2. **크로스 세션 지속성**: 동일한 actor_id + memory_id, 기간별로 다른 session_id로 지식 연속성 활성화
3. **커스텀 메모리 검색 Tool**: AgentCore의 네이티브 search_long_term_memories()를 LlamaIndex FunctionTool로 래핑
4. **시맨틱 처리 파이프라인**: 대화 이벤트 → 시맨틱 메모리 변환을 위한 120초 대기
5. **동적 세션 관리**: 유연한 세션 처리를 위해 memory.context.session_id 사용

**학습 내용:**

- 여러 환자 상담에 걸쳐 지속적인 AgentCore Memory 생성
- 시간 경과에 따른 누적 의료 지식 구축
- 환자 기록 및 치료 결과 전체에 대한 시맨틱 검색 구현
- 치료 효과 및 의료 인사이트 진화 추적
- 크로스 세션 의료 지식 지속성 및 검색 테스트

## 시나리오 컨텍스트

이 예제에서는 수개월 및 수년에 걸친 여러 환자 상담에 걸쳐 의료 지식을 유지하는 "Medical Knowledge Assistant"를 생성합니다. 어시스턴트는 AgentCore Memory를 사용하여 환자 기록, 치료 프로토콜, 약물 상호작용 및 임상 결과의 지속적인 지식 베이스를 구축하며, 이는 시간이 지남에 따라 성장하고 진화하여 정교한 종단적 의료 분석을 가능하게 합니다.

## 사전 요구 사항

- Python 3.10+
- 적절한 권한이 있는 AWS 계정
- AgentCore Memory 권한이 있는 AWS IAM 역할:
  - `bedrock-agentcore:CreateMemory`
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents`
  - `bedrock-agentcore:RetrieveMemories`
- Amazon Bedrock models에 대한 액세스

## 단계 1: 종속성 설치 및 설정

In [ ]:
# 필요한 라이브러리 설치 (semantic strategy toolkit 포함)
%pip install llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse boto3 bedrock-agentcore-starter-toolkit

In [ ]:
# 필요한 컴포넌트 import
from bedrock_agentcore.memory import MemoryClient  # AgentCore Memory 클라이언트
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager  # 메모리 관리자
from bedrock_agentcore.memory.session import MemorySessionManager  # 세션 관리
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies.semantic import SemanticStrategy  # Semantic 전략
from llama_index.memory.bedrock_agentcore import AgentCoreMemory, AgentCoreMemoryContext  # LlamaIndex 메모리 통합
from llama_index.llms.bedrock_converse import BedrockConverse  # Bedrock LLM
from llama_index.core.agent.workflow import FunctionAgent  # LlamaIndex Agent
from llama_index.core.tools import FunctionTool  # Function Tool 래퍼
from datetime import datetime
import os

print("✅ All dependencies imported successfully!")

## 단계 2: AgentCore Memory 구성

장기 의료 지식을 위한 AgentCore Memory 리소스를 생성하거나 가져옵니다:

In [ ]:
# Semantic Strategy를 사용하여 AgentCore Memory 생성
region = os.getenv('AWS_REGION', 'us-east-1')
memory_manager = MemoryManager(region_name=region)

try:
    # 자동 인사이트 추출을 위한 semantic strategy로 메모리 생성
    memory = memory_manager.get_or_create_memory(
        name=f'MedicalAssistantSemantic_{int(datetime.now().timestamp())}',
        strategies=[SemanticStrategy(name="medicalLongTermMemory")],
        event_expiry_days=365  # 의료 기록을 위한 12개월 보존
    )
    memory_id = memory.get('id')
    print(f"✅ Created Semantic Memory: {memory_id}")
    print(f"   Status: {memory.get('status')}")
    print(f"   Strategies: {[s.get('name') if isinstance(s, dict) else str(s) for s in memory.get('strategies', [])]}")
    
    # 메모리가 ACTIVE 상태가 될 때까지 대기
    if memory.get('status') != 'ACTIVE':
        print(f"\n⏳ Waiting for memory to become ACTIVE (currently {memory.get('status')})...")
        import time
        max_wait = 300  # 최대 5분 대기
        waited = 0
        while waited < max_wait:
            time.sleep(10)
            waited += 10
            # 상태 확인
            current_memory = memory_manager.get_memory(memory_id)
            status = current_memory.get('status')
            print(f"   [{waited}s] Status: {status}")
            if status == 'ACTIVE':
                print(f"✅ Memory is now ACTIVE! (took {waited} seconds)")
                break
        else:
            print(f"⚠️  Memory still not ACTIVE after {max_wait}s. Proceeding anyway...")
    
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    memory_id = "your-memory-id-here"  # 기존 memory ID로 교체

## 단계 3: Medical Tools 구현

종단적 의료 분석을 위한 전문 도구를 정의합니다:

In [ ]:
def assess_patient_symptoms(patient_id: str, symptoms: str, severity: str, duration: str) -> str:
    """심각도 및 기간 추적과 함께 환자 증상 평가"""
    return f"🩺 Assessed symptoms for {patient_id} (Severity: {severity}, Duration: {duration})"

def check_drug_interactions(patient_id: str, medications: str, interaction_level: str, recommendations: str) -> str:
    """안전 권장사항과 함께 약물 상호작용 확인"""
    return f"💊 {patient_id} drug interaction check: {interaction_level} - {recommendations}"

def document_treatment_protocol(protocol_type: str, indication: str, effectiveness: str, side_effects: str) -> str:
    """효과 및 부작용과 함께 치료 프로토콜 문서화"""
    print(f"📋 Treatment protocol: {protocol_type} for {indication} (Effectiveness: {effectiveness})")
    return f"Documented treatment protocol: {protocol_type}"

def update_clinical_guideline(patient_id: str, guideline_type: str, recommendation: str, evidence_level: str) -> str:
    """특정 환자에 대한 임상 가이드라인 업데이트"""
    print(f"📖 Clinical guideline: {patient_id} - {guideline_type} ({evidence_level} evidence)")
    return f"Updated guideline for {patient_id}"

def log_treatment_outcome(patient_id: str, treatment: str, outcome: str, follow_up_needed: str) -> str:
    """후속 조치 요구사항과 함께 치료 결과 기록"""
    print(f"🏥 Treatment outcome: {patient_id} - {treatment}: {outcome}")
    return f"Logged outcome for {patient_id}"

def log_medical_milestone(quarter: str, milestone: str, details: str) -> str:
    """분기 및 상세 진행 상황과 함께 의료 마일스톤 기록"""
    print(f"🎯 {quarter} milestone: {milestone}")
    return f"Logged milestone for {quarter}: {milestone} - {details}"

def track_clinical_metrics(metric_type: str, value: str, patient_id: str, quarter: str) -> str:
    """환자 및 타임라인과 함께 특정 임상 지표 추적"""
    print(f"📊 {quarter}: {metric_type} = {value} (for {patient_id})")
    return f"Tracked {metric_type}: {value} for {patient_id} in {quarter}"

def save_medical_insight(insight: str, quarter: str, clinical_context: str) -> str:
    """임상 컨텍스트와 함께 의료 인사이트 저장"""
    print(f"💡 {quarter} insight: {insight[:50]}...")
    return f"Saved {quarter} insight with clinical context: {clinical_context}"

# Agent를 위한 tool 객체 생성
medical_tools = [
    FunctionTool.from_defaults(fn=assess_patient_symptoms),
    FunctionTool.from_defaults(fn=check_drug_interactions),
    FunctionTool.from_defaults(fn=document_treatment_protocol),
    FunctionTool.from_defaults(fn=update_clinical_guideline),
    FunctionTool.from_defaults(fn=log_treatment_outcome),
    FunctionTool.from_defaults(fn=log_medical_milestone),
    FunctionTool.from_defaults(fn=track_clinical_metrics),
    FunctionTool.from_defaults(fn=save_medical_insight)
]

print("✅ Medical tools created!")

## 단계 3b: 메모리 검색 Tool 추가

agent가 장기 메모리를 검색할 수 있도록 하는 도구를 생성합니다:

In [ ]:
def create_memory_retrieval_tool(memory_id: str, actor_id: str, region: str):
    """Agent가 자신의 장기 메모리를 검색할 수 있도록 하는 도구 생성"""
    
    def search_long_term_memory(query: str) -> str:
        """환자, 치료, 프로토콜 및 결과에 대한 관련 의료 정보를 장기 메모리에서 검색.
        
        다음 정보를 회상해야 할 때 이 도구를 사용:
        - 환자 정보 (증상, 치료, 결과)
        - 치료 프로토콜 및 효과
        - 약물 상호작용 및 안전 프로파일
        - 임상 가이드라인 및 권장사항
        - 의료 인사이트 및 학습된 교훈
        
        Args:
            query: 필요한 정보를 설명하는 검색 쿼리 (예: 'PATIENT-001 symptoms', 'diabetes protocols', 'Q1 outcomes')
        
        Returns:
            장기 메모리에서 가져온 관련 정보
        """
        try:
            from bedrock_agentcore.memory.session import MemorySessionManager
            
            # 세션 관리자 생성 (memory_id와 region만 필요)
            session_manager = MemorySessionManager(
                memory_id=memory_id,
                region_name=region
            )
            
            # semantic strategy namespace에서 장기 메모리 검색
            results = session_manager.search_long_term_memories(
                query=query,
                namespace_prefix="/strategies/",  # semantic strategy namespace에서 검색
                top_k=5,
                max_results=10
            )
            
            if not results:
                return "No relevant information found in long-term memory. This might be new information or the memory extraction may still be processing."
            
            # Agent를 위한 결과 포맷팅
            output = "📚 Retrieved from long-term memory:\\n\\n"
            for i, result in enumerate(results, 1):
                # MemoryRecord 객체 - content 속성 접근
                content = getattr(result, 'content', str(result))
                # 매우 긴 콘텐츠 잘라내기
                if len(content) > 300:
                    content = content[:300] + "..."
                output += f"{i}. {content}\\n\\n"
            
            return output
            
        except Exception as e:
            return f"⚠️ Error searching memory: {str(e)}. Proceeding without historical context."
    
    return FunctionTool.from_defaults(fn=search_long_term_memory)

# 메모리 검색 도구 생성
memory_search_tool = create_memory_retrieval_tool(memory_id, "medical-assistant", region)

# tools 리스트에 메모리 검색 추가
medical_tools_with_memory = medical_tools + [memory_search_tool]

print(f"✅ Memory retrieval tool created! Total tools: {len(medical_tools_with_memory)}")
print("   Using namespace: /strategies/ (for semantic strategy compatibility)")

## 단계 3c: 메모리 구성 확인

semantic strategy가 올바르게 구성되었는지 확인합니다:

In [ ]:
# 메모리 구성 확인
memory_info = memory_manager.get_memory(memory_id)
print(f"Strategies: {memory_info.get('strategies')}")
print(f"Status: {memory_info.get('status')}")
print(f"Name: {memory_info.get('name')}")

# Strategy 세부 정보 표시
strategies = memory_info.get('strategies', [])
for strategy in strategies:
    print(f"\nStrategy Details:")
    print(f"  Name: {strategy.get('name')}")
    print(f"  Type: {strategy.get('type')}")
    print(f"  Status: {strategy.get('status')}")
    print(f"  ID: {strategy.get('strategyId')}")

## 단계 4: 멀티 세션 Agent 구현

다양한 의료 기간을 시뮬레이션하는 헬퍼 함수를 생성합니다:

In [ ]:
# 장기 메모리 구성 (크로스 세션)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
ASSISTANT_ID = "medical-assistant"  # 모든 세션에서 동일한 assistant

def create_medical_session(session_name: str):
    """장기 메모리 지속성을 가진 새로운 의료 세션 생성"""
    context = AgentCoreMemoryContext(
        actor_id=ASSISTANT_ID,         # 동일한 assistant
        memory_id=memory_id,         # 동일한 메모리 저장소 (장기 메모리 활성화)
        session_id=f"medical-{session_name}", # 기간별로 다른 세션
        namespace="/medical-analysis/"
    )
    
    memory = AgentCoreMemory(context=context)
    llm = BedrockConverse(model=MODEL_ID)
    agent = FunctionAgent(
        tools=medical_tools_with_memory,  # 메모리 검색 기능이 있는 도구 사용
        llm=llm, 
        verbose=True,  # 메모리 검색 시점을 보기 위해 verbose 활성화
        system_prompt="""You are a senior medical assistant with access to long-term memory.
        
CRITICAL: When asked about patients, treatments, protocols, or historical information, 
you MUST use the search_long_term_memory tool FIRST before responding.

For example:
- "What patients am I treating?" → Use search_long_term_memory("patients treatments")
- "What protocols have I used?" → Use search_long_term_memory("medical protocols")
- "What outcomes have I achieved?" → Use search_long_term_memory("patient outcomes")

Always provide conclusive, complete responses without asking follow-up questions.\n
Execute all requested actions immediately and completely. Provide detailed, professional responses."""
    )
    
    return agent, memory

print("✅ Multi-session Medical Knowledge Assistant setup complete!")

## 단계 5: Q1 의료 세션 - 초기 환자 평가

첫 번째 의료 세션을 시작하고 환자 기준선을 설정합니다:

In [ ]:
# === Q1 의료 세션 ===
print("🗓️ === Q1: INITIAL PATIENT ASSESSMENT ===")

agent_q1, memory_q1 = create_medical_session("q1")

# 초기 환자 증상 평가
response = await agent_q1.run(
    "I'm Senior Medical Assistant Dr. Maria Rodriguez. Assess patient symptoms for 'PATIENT-001' with symptoms 'fatigue, increased thirst, frequent urination, blurred vision', "
    "severity 'Moderate', duration '3 weeks'.",
    memory=memory_q1
)

print("🎯 Q1 Initial Assessment:")
print(response)

In [ ]:
# 초기 임상 가이드라인 문서화
response = await agent_q1.run(
    "Update clinical guideline for 'PATIENT-001': guideline type 'Diabetes Management', "
    "recommendation 'initiate metformin 500mg BID, lifestyle counseling, glucose monitoring', evidence level 'high'.",
    memory=memory_q1
)
print("💭 Q1 Clinical Guideline:", response)

response = await agent_q1.run(
    "Update clinical guideline for 'PATIENT-001': guideline type 'Lifestyle Modification', "
    "recommendation 'dietary consultation, exercise program, weight management target 10% reduction', evidence level 'high'.",
    memory=memory_q1
)
print("💭 Q1 Lifestyle Guideline:", response)
# 의료 소견 명시적으로 추적
await agent_q1.run(
    "Save medical finding: finding 'Patient responds well to treatment protocol', confidence 'high'.",
    memory=memory_q1
)


In [ ]:
# 의료 메모리 추출을 위한 대기 시간 허용
import asyncio
print("\n⏳ Waiting for medical memory extraction...")
await asyncio.sleep(120)  # Semantic 메모리 처리를 위한 대기
print("✅ Medical memory processing complete")


In [ ]:
# 이벤트가 저장되었는지 확인
print("\n🔍 Verifying events were stored...")
try:
    client = MemoryClient(region_name=region)
    events = client.list_events(
        memory_id=memory_id,
        actor_id="medical-assistant",  # 도메인별 ID로 교체됨
        session_id=memory_q1.context.session_id  # 동적 세션 ID
    )
    print(f"✅ Stored {len(events)} conversational events in session")
except Exception as e:
    print(f"⚠️  Could not verify events: {e}")

# semantic 메모리 처리를 위한 대기 시간 허용
import asyncio
print("\n⏳ Waiting for semantic memory extraction and indexing...")
print("   (AgentCore processes conversational events in the background)")
await asyncio.sleep(120)  # 메모리 추출을 위한 대기 시간 증가
print("✅ Memory processing complete - memories should now be searchable")

## 단계 6: Q2 의료 세션 - 치료 프로토콜 업데이트

장기 메모리 검색을 테스트하고 치료 반응에 적응합니다:

In [ ]:
# === Q2 의료 세션 ===
print("\n🗓️ === Q2: TREATMENT PROTOCOL UPDATE (NEW SESSION) ===")

agent_q2, memory_q2 = create_medical_session("q2")

# 크로스 세션 환자 회상 테스트 - agent는 search_long_term_memory 도구를 사용해야 함
print("\n🧠 Testing memory retrieval across sessions...")
print("   (Watch for the agent to use search_long_term_memory tool)\n")

response = await agent_q2.run(
    "What patients am I treating? What are their symptoms, treatments, and clinical guidelines?",
    memory=memory_q2
)

print("\n🧠 Q2 Patient Recall:")
print(response)
print("\n✅ Expected: PATIENT-001, diabetes symptoms, metformin treatment, lifestyle guidelines")

In [ ]:
# 치료 프로토콜 문서화
response = await agent_q2.run(
    "Document treatment protocol: protocol type 'Type 2 Diabetes Management', "
    "indication 'newly diagnosed T2DM with HbA1c 8.2%', "
    "effectiveness 'HbA1c reduced to 7.1% after 3 months', "
    "side effects 'mild GI upset initially, resolved with food intake'.",
    memory=memory_q2
)
print("🌍 Q2 Treatment Protocol:", response)

# 반응에 기반한 임상 가이드라인 업데이트
response = await agent_q2.run(
    "Update clinical guideline for 'PATIENT-001': guideline type 'Medication Adjustment', "
    "recommendation 'continue metformin 500mg BID, add glipizide 5mg daily for additional glycemic control', evidence level 'high'.",
    memory=memory_q2
)
print("⚖️ Q2 Guideline Update:", response)

In [ ]:
# Q2 임상 지표 추적
response = await agent_q2.run(
    "Track clinical metrics for 'PATIENT-001': metric type 'HbA1c Improvement', value '8.2% to 7.1%', "
    "patient_id 'PATIENT-001', quarter 'Q2 2024'.",
    memory=memory_q2
)
print("📈 Q2 Clinical Metrics:", response)

# 치료 비교 테스트
response = await agent_q2.run(
    "How did PATIENT-001's treatment progress from Q1 to Q2? Compare initial vs current approach.",
    memory=memory_q2
)
print("📊 Q2 Treatment Analysis:")
print(response)
print("\n✅ Expected: Q1 metformin monotherapy → Q2 combination therapy, HbA1c improvement")

## 단계 7: Q3 의료 세션 - 합병증 관리

합병증 관리 및 신규 환자 접수로 진행합니다:

In [ ]:
# === Q3 의료 세션 ===
print("\n🗓️ === Q3: COMPLICATION MANAGEMENT AND NEW PATIENT ===")

agent_q3, memory_q3 = create_medical_session("q3")

# 치료 결과 기록
response = await agent_q3.run(
    "Log treatment outcome for 'PATIENT-001' with treatment 'Diabetes Management Protocol', "
    "outcome 'Target HbA1c achieved (6.8%), weight loss 15 lbs, BP controlled', "
    "follow_up_needed 'quarterly HbA1c monitoring, annual eye exam, continue current regimen'.",
    memory=memory_q3
)
print("📅 Q3 Treatment Outcome:", response)

# 신규 환자 평가 시작
response = await agent_q3.run(
    "Assess patient symptoms for 'PATIENT-002': symptoms 'chest pain, shortness of breath, palpitations', "
    "severity 'High', duration '2 days'.",
    memory=memory_q3
)
print("💭 Q3 New Patient Assessment:", response)

In [ ]:
# 포괄적인 의료 기록 회상 테스트
response = await agent_q3.run(
    "What is the complete medical care history? Include all patients, treatments, "
    "protocols, and outcomes.",
    memory=memory_q3
)
print("📋 Q3 Complete History:")
print(response)
print("\n✅ Expected: PATIENT-001 diabetes journey → PATIENT-002 cardiac assessment, protocol evolution")

## 단계 8: Q4 의료 세션 - 연말 검토 및 계획

시맨틱 검색 및 연간 의료 분석을 테스트합니다:

In [ ]:
# === Q4 의료 세션 ===
print("\n🗓️ === Q4: YEAR-END REVIEW AND PLANNING ===")

agent_q4, memory_q4 = create_medical_session("q4")

# 연간 의료 지표 추적
response = await agent_q4.run(
    "Track clinical metrics: metric type '2024 Annual Performance', value 'Patients treated: 2, Diabetes control achieved: 100%, Complications prevented: 1', "
    "patient_id 'ANNUAL-SUMMARY', quarter '2024 Annual'.",
    memory=memory_q4
)
print("📈 Q4 Annual Metrics:", response)

# 치료 프로토콜 상관관계 테스트
response = await agent_q4.run(
    "What treatment protocols have I documented this year? How effective were they?",
    memory=memory_q4
)
print("🌍 Q4 Protocol Effectiveness Analysis:")
print(response)
print("\n✅ Expected: Diabetes management protocol → successful HbA1c control")

In [ ]:
# 유사한 의료 접근법에 대한 시맨틱 검색 테스트
response = await agent_q4.run(
    "What clinical guidelines have I used? Which were most effective based on patient outcomes?",
    memory=memory_q4
)
print("⚖️ Q4 Guideline Effectiveness Analysis:")
print(response)
print("\n✅ Expected: Diabetes management + lifestyle modification = successful outcomes")

## 단계 9: 2년차 Q1 세션 - 다년간 의료 관점

장기 의료 지식 및 실무 진화를 테스트합니다:

In [ ]:
# === 2년차 Q1 의료 세션 ===
print("\n🗓️ === YEAR 2 Q1: MULTI-YEAR MEDICAL PERSPECTIVE ===")

agent_y2q1, memory_y2q1 = create_medical_session("year2-q1")

# 다년간 의료 실무 분석
response = await agent_y2q1.run(
    "Analyze my medical practice evolution: How have my patients and treatments developed over the past year? "
    "What were the key clinical decisions and their outcomes?",
    memory=memory_y2q1
)
print("📊 Year 2 Q1 Practice Analysis:")
print(response)
print("\n✅ Expected: PATIENT-001 → PATIENT-002 progression, protocol refinement, outcome improvement")

In [ ]:
# 임상 프로토콜 진화 추적 테스트
response = await agent_y2q1.run(
    "How have my clinical protocols and guidelines evolved? What treatments have I applied and why?",
    memory=memory_y2q1
)
print("💭 Year 2 Q1 Protocol Evolution:")
print(response)
print("\n✅ Expected: Started with diabetes protocols → expanded to cardiac care")

## 단계 10: 최종 의료 실무 평가

장기 의료 분석 기능에 대한 포괄적인 테스트:

In [ ]:
# 최종 포괄적인 의료 실무 쿼리
response = await agent_y2q1.run(
    "Provide my complete medical practice portfolio: all patients with their clinical journeys, "
    "treatment effectiveness, protocol applications, guideline utilization, and patient outcomes. "
    "Include lessons learned and best practices developed.",
    memory=memory_y2q1
)
print("💼 Complete Medical Practice Portfolio:")
print(response)
print("\n✅ Expected: Full patient portfolio with treatment evolution, protocol effectiveness, and outcome analysis")

## 🧪 자동화된 테스트 검증
메모리 통합이 올바르게 작동하는지 검증하기 위해 이 셀을 실행하세요:

In [ ]:
# 검증 함수를 인라인으로 정의
class TestValidator:
    def __init__(self):
        self.results = {}
    
    def validate_memory_recall(self, response):
        """Agent가 세션 초반의 정보를 회상할 수 있는지 확인"""
        # 실질적인 응답 확인 (단순히 "I don't know"가 아닌)
        has_content = len(response) > 50
        # 메모리 지표 확인
        has_memory_indicators = any(word in response.lower() for word in 
            ['earlier', 'mentioned', 'discussed', 'previously', 'you', 'we', 'our'])
        return "✅ PASS" if (has_content and has_memory_indicators) else "❌ FAIL"
    
    def validate_session_memory(self, response):
        """Agent가 세션 내에서 컨텍스트를 유지하는지 확인"""
        has_memory_content = len(response) > 100 and any(word in response.lower() for word in 
            ['previous', 'earlier', 'mentioned', 'discussed', 'before', 'already'])
        return "✅ PASS" if has_memory_content else "❌ FAIL"
    
    def validate_cross_reference(self, response):
        """Agent가 현재 쿼리를 이전 컨텍스트와 연결할 수 있는지 확인"""
        # 연결 언어 찾기
        connecting_words = ['relate', 'connection', 'previous', 'earlier', 'discussed', 
                           'mentioned', 'context', 'based on', 'as we', 'as i']
        has_connection = any(word in response.lower() for word in connecting_words)
        has_substance = len(response) > 80
        return "✅ PASS" if (has_connection and has_substance) else "❌ FAIL"
    
    def run_validation_summary(self, test_results):
        print("🧪 COMPREHENSIVE TEST VALIDATION SUMMARY")
        print("=" * 60)
        
        total_tests = len(test_results)
        passed_tests = sum(1 for result in test_results.values() if "PASS" in result)
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        for test_name, result in test_results.items():
            print(f"{test_name}: {result}")
        
        print("=" * 60)
        print(f"📊 Overall Pass Rate: {passed_tests}/{total_tests} ({pass_rate:.1f}%)")
        
        if pass_rate >= 80:
            print("✅ EXCELLENT: Memory integration working correctly!")
        elif pass_rate >= 60:
            print("⚠️  GOOD: Most memory features working, some issues to investigate")
        else:
            print("❌ NEEDS ATTENTION: Memory integration has significant issues")
        
        return pass_rate

validator = TestValidator()
print("✅ Validation functions loaded!")

In [ ]:
# 모든 검증 테스트 실행
test_results = {}

# 테스트 1: 메모리 회상 - agent가 논의된 내용을 회상할 수 있는가?
response1 = await agent_y2q1.run("What have we discussed so far in this session?", memory=memory_y2q1)
test_results['Memory Recall'] = validator.validate_memory_recall(str(response1))
print(f"Response 1 length: {len(str(response1))} chars")

# 테스트 2: 세션 메모리 - agent가 컨텍스트를 유지하는가?
response2 = await agent_y2q1.run("What did we talk about earlier?", memory=memory_y2q1)
test_results['Session Memory'] = validator.validate_session_memory(str(response2))
print(f"Response 2 length: {len(str(response2))} chars")

# 테스트 3: 상호 참조 기능 - 이전 컨텍스트와 연결할 수 있는가?
response3 = await agent_y2q1.run("How does this relate to what we discussed before?", memory=memory_y2q1)
test_results['Cross Reference'] = validator.validate_cross_reference(str(response3))
print(f"Response 3 length: {len(str(response3))} chars")

# 결과 표시
validator.run_validation_summary(test_results)

## 요약

이 노트북에서는 다음을 시연했습니다:

✅ **장기 메모리 통합**: 크로스 세션 의료 분석을 위해 LlamaIndex와 함께 AgentCore Memory 사용

✅ **환자 치료 추적**: 여러 분기에 걸친 환자 진화 및 치료 개발

✅ **임상 인텔리전스**: 치료 프로토콜 및 적용에 대한 시맨틱 검색

✅ **의료 프로토콜 진화**: 초기 평가에서 증거 기반 접근 방식으로의 자연스러운 진행

✅ **치료 효과**: 임상 결정 및 환자 결과에 대한 상세한 추적

✅ **의료 실무 우수성**: 시간 경과에 따른 포괄적인 환자 관리 및 치료 최적화

Medical Knowledge Assistant는 장기 메모리가 어시스턴트를 시간이 지남에 따라 더 똑똑해지는 지속적인 의료 파트너로 변환하여 완전한 환자 기록을 유지하고 확장된 치료 기간에 걸쳐 정교한 임상 지식 검색을 가능하게 하는 방법을 보여줍니다.

## 정리

이 노트북에서 사용된 리소스를 정리하기 위해 메모리를 삭제하겠습니다:

**참고**: 메모리를 영구적으로 삭제하려는 경우에만 실행하세요. memory_id 변수는 이 노트북의 앞부분에서 생성된 메모리의 ID를 포함해야 합니다.

In [ ]:
# AgentCore Memory 리소스 정리
try:
    from bedrock_agentcore.memory import MemoryClient
    
    client = MemoryClient(region_name=region)
    client.delete_memory(memory_id)
    print(f"✅ Successfully deleted memory: {memory_id}")
    
except NameError as e:
    print(f"⚠️  Variable not defined: {e}")
    print("Run the notebook from the beginning or set variables manually:")
    print("# memory_id = 'your-memory-id-here'")
    print("# region = 'us-east-1'")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")